# Simple ETL Application

In this notebook, we connect to an api, extract data, create a spark dataframe and then finally write to a postgres database. 

In [1]:
import requests
import json
import pandas as pd
import psycopg2
from src.get_data import get_data
from src.sql_base import sql_base

## Prepare database

In [5]:
db = sql_base()

In [8]:
db.create_schema(schema_name="test_table")

In [7]:
db.create_table(table_name="test",table_definitions="name text, surname text,age numeric(2)")

## Create table in postgres Database using psycopg2 library

In [8]:
query = """CREATE TABLE test_table (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    email TEXT UNIQUE
)"""

#query = f"CREATE SCHEMA IF NOT EXISTS TEST"

In [9]:
# url
url = "http://postgres-db:45432/postgres_db"
#user
username = "postgres"
password = "postgres"

conn_details = psycopg2.connect(
    host='postgres-db',
    database="postgres",
    user=username,
    # secretlint-disable
    password=password,
    # secretlint-enable
    port='5432'
)

with conn_details:
    cursor = conn_details.cursor()
    query = query
    cursor.execute(query)

DuplicateTable: relation "test_table" already exists


In [12]:
query = """(select * from test_table)"""

In [14]:
print(query_runner(query))

None


## Insert Records

## Create a spark Session

In [39]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.extraClassPath", "postgresql-connector-java.jar") \
    .config("spark.executor.extraClassPath", "postgresql-connector-java.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 21:19:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create Spark dataframe

In [40]:
spark_df = spark.createDataFrame(data)

## Test Functions 

In [43]:
import pytest

def test_test_data():
    """
    Tests API is returning values
    """
    status_code,_ = get_data()
    
    assert status_code == 200



In [53]:
! pytest

============================= test session starts ==============================
platform linux -- Python 3.11.9, pytest-8.1.1, pluggy-1.4.0
rootdir: /app
plugins: anyio-4.3.0
collected 1 item                                                               

tests/test_get_data.py .                                                 [100%]

============================== 1 passed in 1.34s ===============================


In [20]:
names = {'1':"mukovhe"}
         

In [21]:
for i in names:
    print(i)

1


In [22]:
names['1']

'mukovhe'